In [349]:
import pandas as pd
import pickle
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import text_processor as tp

In [145]:
sia = SentimentIntensityAnalyzer()

In [295]:
# import classifier
with open('pickles/amazon/mnb_classifier.pickle', 'rb') as data:
    model = pickle.load(data)
    
# import tfidf
with open('pickles/amazon/tfidf.pickle', 'rb') as data:
     tfidf = pickle.load(data)

In [198]:
sentiment_map = {'Negative':0, 'Positive':1, 'Compliant':2}

In [297]:
def get_sentiment(text):
    """
    Predicts the sentiment of text using the Multinomial Naive Bayes Model
    """
    sentiment_id = model.predict(tfidf.transform([text]))
    return get_name(sentiment_id)

In [300]:
def get_name(sentiment_id):
    """
    Gets sentiment name from sentiment_map using sentiment_id
    """
    for sentiment, id_ in sentiment_map.items():
        if id_ == sentiment_id:
            return sentiment

In [301]:
def get_noun(text):
    """
    Finds noun of the text
    """
    tokenizer = ToktokTokenizer()
    tokens = tokenizer.tokenize(text)    
    pos_tags = nltk.pos_tag(tokens)
    nouns = []
    for word, tag in pos_tags:
        if tag == "NN" or tag == "NNP" or tag == "NNS":
            nouns.append(word)
    return nouns
    

In [302]:
def top_pos_word(text):
    """
    Finds top positive word using nltk vader library
    """
    pos_polarity = dict()
    for word in tp.get_tokens(text):
        pos_score = sia.polarity_scores(word)['pos']
        if word not in pos_polarity:
            pos_polarity[word] = pos_score
        else:
            pos_polarity[word] += pos_score
    top_word = max(pos_polarity, key=pos_polarity.get)
    return top_word

In [303]:
def top_neg_word(text):
    """
    Finds top negative word using nltk vader library
    """
    neg_polarity = dict()
    for word in tp.get_tokens(text):
        neg_score = sia.polarity_scores(word)['neg']
        if word not in neg_polarity:
            neg_polarity[word] = neg_score
        else:
            neg_polarity[word] += neg_score
    top_word = max(neg_polarity, key=neg_polarity.get)
    return top_word

In [304]:
def sentiment_analysis(text):
    """
    Finds the sentiment of text, prints positive or negative word and 
    prints the causing words of positivity or negativity
    """
    text = tp.clean_text(text)
    sentiment = get_sentiment(text)
    print(f'Sentiment: {sentiment}')
    if sentiment == 'Positive':
        nouns = get_noun(text)
        print(f'Positive word: {top_pos_word(text)}')
        print(f'Cause of positivity: {nouns}')
    elif sentiment == 'Negative':
        nouns = get_noun(text)
        print(f'Negative word: {top_neg_word(text)}')
        print(f'Cause of negativity: {nouns}')

### Test the model with some sample input texts

In [321]:
text1 = "It's beyond my expectation, and it can even show music score. Not fast turning though."

In [322]:
sentiment_analysis(text1)

Sentiment: Positive
Positive word: beyond
Cause of positivity: ['expectation', 'music', 'score', 'turn']


In [323]:
text2 = "Everything is great except that i can't read external pdfs properly. Either it zooms too much of too little. For kindle books it provides great experience. Battery life is awesome!"

In [324]:
sentiment_analysis(text2)

Sentiment: Positive
Positive word: great
Cause of positivity: ['everything', 'pdfs', 'zoom', 'book', 'experience', 'battery', 'life', 'awesome']


In [325]:
text3 = "it would not load my books proper. took a dozen tries erasing an dreregistering. screen too dark"

In [326]:
sentiment_analysis(text3)

Sentiment: Negative
Negative word: would
Cause of negativity: ['book', 'take', 'dozen', 'erase', 'dark']


In [330]:
text4 = "For the first month it was good. But from the starting i was having blue tooth connectivity issue. It was not even having good connection within 5metres. Then within 30 days due to sweating it completely stopped working. I am still searching for the right wireless ear phone which can be used for long run, marathon and resist sweating.Very poor customer care service. I returned the product and have to wait for 3 week for dispatching a new one. Still on 4th week no shipment done."

In [331]:
sentiment_analysis(text4)

Sentiment: Compliant


In [332]:
text5 ="All of them quit working. There's absolutely no way to revive them once that battery loses power If NOT charged immediately, if even just a few weeks go by, NO HOPE of ever using your Kindle again."

In [333]:
sentiment_analysis(text5)

Sentiment: Negative
Negative word: lose
Cause of negativity: ['quit', 'work', 'way', 'battery', 'power', 'charge', 'week']


In [347]:
text6 = "My product was damaged at the 7th day and I initiated the replacement option, the reverse pick up was scheduled however no delivery information is available still. Itz being 3 days now, when you go to boat website the tracking status button does not works on the website and even you cannot reach an representative via call. I don't know if I am gonna receive the product back or not!"

In [348]:
sentiment_analysis(text6)

Sentiment: Compliant
